### Import pacchetti

In [ ]:
import json
import nltk
import random
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import bokeh
import bokeh.models
import bokeh.plotting
import os
import pandas as pd
import tensorflow.compat.v2 as tf
import tensorflow_hub as hub
from tensorflow_text import SentencepieceTokenizer
import sklearn.metrics.pairwise

from simpleneighbors import SimpleNeighbors
from tqdm import tqdm
from tqdm import trange

from google.colab import files
from nltk import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Caricamento risorse

In [ ]:
corpus = list()

files.upload() #file mio
files.upload() #file Lore
files.upload() #file Ludo

carica_file = list()
carica_lore = list()
carica_ludo = list()

#file mio
with open('file_Stefano.txt', 'r', encoding='utf-8') as f:
  for line in f:
    carica_file.append(line)

#file Lore
with open('file_Lorenzo.txt', 'r', encoding='utf-8') as lo:
  articoliSeparati = json.loads(lo.read())
  for articolo in articoliSeparati:
    for frase in articolo:
        carica_lore.append(frase)

#file Ludo
with open('q&a.json', 'r') as ludo:
    q_a = json.load(ludo)
    for k in q_a.keys():
      carica_ludo.append(k)
  
corpus = carica_file + carica_lore + carica_ludo

### Preprocessing

In [ ]:
# SENTENCE SPLITTING
doc_corpus = list()
sentence_corpus = list()

for line in corpus:
  sentence_doc_list = nltk.sent_tokenize(line)
  doc_corpus.append(sentence_doc_list)

for doc in doc_corpus:
  for sentence in doc:
    ######### se ci sono delle domande, non le teniamo in conto####
    ######### NON LO POSSO FARE QUI ###########
    #if sentence[len(sentence)-1] != '?':
    sentence_corpus.append(sentence.lower())

In [ ]:
# ogni line abbiamo un documento
for l in corpus[:10]:
  print(l)

La pandemia da COVID-19 ha reso necessaria una riorganizzazione dei percorsi assistenziali legati alla nascita. Per questo motivo, l’ISS mette a disposizione del personale sanitario una selezione di risorse, letteratura ed evidence disponibili su gravidanza, parto e allattamento. In evidenza:  la pagina dedicata alla vaccinazione contro il COVID-19 in gravidanza e allattamentola pagina dedicata allo studio “COVID-19 e salute mentale perinatale: impatto del COVID-19 sul vissuto e lo stato emotivo in epoca perinatale delle donne in contatto con i Consultori Familiari (CF)”, di cui l’Istituto Superiore di Sanità è coordinatore, aggiornata al 7 gennaio 2021Leggi gli altri approfondimenti:Per aiutare il personale sanitario nel lavoro quotidiano di assistenza alle donne durante la gravidanza, il parto e l’allattamento, le ricercatrici dell’ISS hanno messo a punto alcuni materiali di comunicazione specifici. Visita lapagina dedicata.Data di ultimo aggiornamento: 9 gennaio 2021 Data di creazio

In [ ]:
# ogni line abbiamo una lista di frasi
for l in doc_corpus[:10]:
  print(l)

['La pandemia da COVID-19 ha reso necessaria una riorganizzazione dei percorsi assistenziali legati alla nascita.', 'Per questo motivo, l’ISS mette a disposizione del personale sanitario una selezione di risorse, letteratura ed evidence disponibili su gravidanza, parto e allattamento.', 'In evidenza:  la pagina dedicata alla vaccinazione contro il COVID-19 in gravidanza e allattamentola pagina dedicata allo studio “COVID-19 e salute mentale perinatale: impatto del COVID-19 sul vissuto e lo stato emotivo in epoca perinatale delle donne in contatto con i Consultori Familiari (CF)”, di cui l’Istituto Superiore di Sanità è coordinatore, aggiornata al 7 gennaio 2021Leggi gli altri approfondimenti:Per aiutare il personale sanitario nel lavoro quotidiano di assistenza alle donne durante la gravidanza, il parto e l’allattamento, le ricercatrici dell’ISS hanno messo a punto alcuni materiali di comunicazione specifici.', 'Visita lapagina dedicata.Data di ultimo aggiornamento: 9 gennaio 2021 Data

In [ ]:
# SENTENCE_CORPUS
# unica lista contenente tutte le frasi del corpus
sentence_corpus[:10]

['la pandemia da covid-19 ha reso necessaria una riorganizzazione dei percorsi assistenziali legati alla nascita.',
 'per questo motivo, l’iss mette a disposizione del personale sanitario una selezione di risorse, letteratura ed evidence disponibili su gravidanza, parto e allattamento.',
 'in evidenza:  la pagina dedicata alla vaccinazione contro il covid-19 in gravidanza e allattamentola pagina dedicata allo studio “covid-19 e salute mentale perinatale: impatto del covid-19 sul vissuto e lo stato emotivo in epoca perinatale delle donne in contatto con i consultori familiari (cf)”, di cui l’istituto superiore di sanità è coordinatore, aggiornata al 7 gennaio 2021leggi gli altri approfondimenti:per aiutare il personale sanitario nel lavoro quotidiano di assistenza alle donne durante la gravidanza, il parto e l’allattamento, le ricercatrici dell’iss hanno messo a punto alcuni materiali di comunicazione specifici.',
 'visita lapagina dedicata.data di ultimo aggiornamento: 9 gennaio 2021 d

In [ ]:
token_corpus[:10]

[['pandemia',
  'covid-19',
  'reso',
  'necessaria',
  'riorganizzazione',
  'percorsi',
  'assistenziali',
  'legati',
  'nascita',
  '.'],
 ['motivo',
  ',',
  '’',
  'iss',
  'mette',
  'disposizione',
  'personale',
  'sanitario',
  'selezione',
  'risorse',
  ',',
  'letteratura',
  'evidence',
  'disponibili',
  'gravidanza',
  ',',
  'parto',
  'allattamento',
  '.'],
 ['evidenza',
  ':',
  'pagina',
  'dedicata',
  'vaccinazione',
  'covid-19',
  'gravidanza',
  'allattamentola',
  'pagina',
  'dedicata',
  'studio',
  '“',
  'covid-19',
  'salute',
  'mentale',
  'perinatale',
  ':',
  'impatto',
  'covid-19',
  'vissuto',
  'stato',
  'emotivo',
  'epoca',
  'perinatale',
  'donne',
  'contatto',
  'consultori',
  'familiari',
  '(',
  'cf',
  ')',
  '”',
  ',',
  '’',
  'istituto',
  'superiore',
  'sanità',
  'coordinatore',
  ',',
  'aggiornata',
  '7',
  'gennaio',
  '2021leggi',
  'altri',
  'approfondimenti',
  ':',
  'aiutare',
  'personale',
  'sanitario',
  'lavoro'

### Costruzione del modello - Universal Sentence Encoder Multilingual



In [ ]:
%%capture
#@title Setup Environment
# Install the latest Tensorflow version.
!pip install tensorflow_text
!pip install bokeh
!pip install simpleneighbors[annoy]
!pip install tqdm

Il modello è stato preso da https://tfhub.dev/google/universal-sentence-encoder-multilingual/3

In [ ]:
module_url = 'https://tfhub.dev/google/universal-sentence-encoder-multilingual/3' #@param ['https://tfhub.dev/google/universal-sentence-encoder-multilingual/3', 'https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3']
model = hub.load(module_url)

A differenza di word2vec questo modello non richiede che il testo sia processato (gli passiamo la lista delle frasi del nostro corpus di documenti)

In [ ]:
sentence_embeddings = model(corpus)

In [ ]:
# funzione che calcola la cosine similarity
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

### Impostazione del chatbot

In [ ]:
# function to return a random greeting responde to a users greeting
def greeting_response(text):
    text = text.lower()
    
    # bots greeting response
    bot_greetings = ['ciao', 'ehi']
    # user's greeting
    user_greetings = ['ciao']
    
    # se una parola dell'utente contiene una parola in user_greetings
    for word in text.split():
        if word in user_greetings:
            # restituisci una parola presa a caso da bot_greetings
            return random.choice(bot_greetings)

In [ ]:
def bot_response(user_input, model,embeddings, q_a, corpus):
    input_emb = model([user_input])[0]
    answers_list = list()
    bot_response = ''
    # flag per vedere se esiste una risposta per la query dell'utente
    response_flag = 0
    found_faq = False
    for i in range(len(embeddings)):
      sent = corpus[i]
      # calcoliamo la similarità tra la query e ogni
      # frase (embedding) del corpus
      sim = cosine(input_emb, embeddings[i])
      if sim > 0.05:
        answers_list.append(((sim, sent)))
      i += 1
    # se esistono delle risposte
      # riordiniamo la lista delle possibili risposte a seconda
      # dello score di similarità 
    answers_list.sort(key=lambda x : x[0], reverse=True)
    # prima controlliamo se ci sono delle FAQ
    for k in q_a.keys():
      if answers_list[0][1] == k.lower():
        bot_response = bot_response + q_a[k][0]
        response_flag = 1
        found_faq = True
        return bot_response
   
    i = 0
    quest = True
    if not found_faq:
      while quest:
        if '?' in answers_list[i][1]:
          i += 1
        else:
          bot_response = answers_list[i][1]
          response_flag = 1
          quest = False
    # se non abbiamo trovato una risposta
    if response_flag == 0:
        bot_response = bot_response + 'Scusa, non sono in grado di risponderti.'
    return bot_response

In [ ]:
# esempio di conversazione
print('Co-bot:')
print('Benvenuto! Risponderò a tutti i tuoi dubbi riguardo il coronavirus')
print()

exit_list = ['esci']
while(True):
    print()
    print('User:')
    user_input = input()
    if user_input.lower() in exit_list:
        print('Co-bot: Ciao, a presto!')
        break
    else:
        if greeting_response(user_input) != None:
            print('Co-bot:')
            print(greeting_response(user_input))
        else:
            print('Co-bot: ... ')
            print(bot_response(user_input, model, sentence_embeddings ,q_a, sentence_corpus))

## test domande

In [ ]:
lista_domande = ["Cosa fare nel caso in cui si manifestano i sintomi?", 
            "Posso viaggiare se ho il coronavirus?", 
            "Le donne in gravidanza possono trasmettere la malattia al figlio?", 
            "Quanto tempo resta nel corpo il coronavirus?", 
            "Come funziona il contact tracing?", 
            "Quando finirà la pandemia?", 
            "Il vaccino ha effetti collaterali?", 
            "Cosa sappiamo della nuova variante inglese?", 
            "Quanti tipi di vaccino esistono?", 
            "Fino a quando saranno chiuse le attività commerciali?", 
            "Come distinguiamo un'influenza stagionale da un'influenza covid?", 
            "Chi si è vaccinato può trasmette la malattia?", 
            "Quali sono i colori che possono assumere le regioni?", 
            "Come fanno le persone infette a trasmettere il virus?", 
            "Se vengo dall'estero devo fare il tampone?", 
            "Cos’è l'indice Rt?", 
            "Posso ammalarmi due volte di covid?", 
            "Quanto tempo dura l'isolamento fiduciario?", 
            "È possibile fare sport di contatto?", 
            "Come faccio a capire se sono positiva ma asintomatica?", 
            "Sono stato a contatto con un infetto: cosa devo fare?", 
            "Fino a quando resteranno chiuse le piscine e le palestre?", 
            "Dopo il vaccino, devo comunque continuare a usare la mascherina?", 
            "Che cosa devo fare se voglio andare all'estero?", 
            "Lo smog/inquinamento influisce sulla diffusione del coronavirus/covid?", 
            "Qual è il periodo di copertura del vaccino?", 
            "Se sono all’estero, cosa devo fare per tornare in Italia?", 
            "Come proteggere le persone più anziane dal virus?", 
            "Il coronavirus è pericoloso per gli animali domestici?", 
            "La mascherina è obbligatoria all’aperto?", 
            "Fino a quando sarà adottata la didattica a distanza per le scuole superiori?", 
            "Qual è l'incidenza del covid nei bambini?", 
            "I guanti servono a proteggere dal contagio?", 
            "Si può viaggiare fuori comune in area gialla?", 
            "Quanto sono importanti il distanziamento e la chiusura delle attività nella trasmissione del covid?", 
            "Posso fare attività sportiva se sono positivo asintomatico?", 
            "Per i bambini è obbligatorio l’uso della mascherina?", 
            "Come agiscono i linfociti T nelle infezioni da covid?", 
            "Posso visitare i miei nonni?", 
            "È possibile andare a pesca?", 
            "Quali sono i sintomi del coronavirus?", 
            "Il virus può rimanere sulle superfici?"]

In [ ]:
print('Co-bot:')
print('Benvenuto! Risponderò a tutti i tuoi dubbi riguardo il coronavirus')
print()

exit_list = ['esci']
i=1
for sentence in lista_domande:
    print('\n')
    print('user'+" "+str(i))
    i+=1
    print (sentence)
    user_input = sentence
    if user_input.lower() in exit_list:
        print('Co-bot: Ciao, a presto!')
        break
    else:
        if greeting_response(user_input) != None:
            print('Co-bot:')
            print(greeting_response(user_input))
        else:
            print('Co-bot: ... ')
            print(bot_response(user_input, model, sentence_embeddings,q_a, sentence_corpus))

Co-bot:
Benvenuto! Risponderò a tutti i tuoi dubbi riguardo il coronavirus



user 1
Cosa fare nel caso in cui si manifestano i sintomi?
Co-bot: ... 
dal 2004 il ministero della salute e il centro nazionale per la prevenzione e il controllo delle malattie (ccm), in collaborazione con le regioni, promuovono la progettazione, sperimentazione e l’implementazione di sistemi di sorveglianza di popolazione per raccogliere informazioni sullo stato di salute della popolazione e sui fattori di rischio modificabili e associati all’insorgenza di malattie croniche, al fine di guidare a livello locale le azioni di prevenzione valutarne l’efficacia nel tempo verso gli obiettivi di salute fissati nei piani nazionali e regionali della prevenzione.


user 2
Posso viaggiare se ho il coronavirus?
Co-bot: ... 
la pandemia di covid-19 ha portato con sé vari problemi aggiuntivi o ha aggravato condizioni di vita già difficili per le popolazioni migranti.


user 3
Le donne in gravidanza possono trasmettere la